In [1]:
import pandas as pd

In [2]:
from scipy.stats import chi2_contingency
from scipy.stats import chi2

In [3]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
from sklearn.metrics import classification_report

In [6]:
complaintsData = pd.read_csv('complaint_data.csv')

In [7]:
complaintsData.shape

(309260, 16)

In [8]:
complaintsData.drop_duplicates().shape

(309260, 16)

In [9]:
complaintsData.head()

,Extract Run Date,UniqueRandomComplaintId,Allegation Id,Allegation FADO Type,Allegation,Case Type,Closed Year,Days From Received Date To Case Closing,Re-opened,Ever On DA Hold,Complaint Outcome Path,Complaint Disposition,Allegation Outcome Path,Allegation Disposition,Borough Of Incident,Patrol Borough Of Incident
0,05/25/2018,1,1,Force,Beat,CCRB,2000,2817,False,False,Full Investigation,Unsubstantiated,Full Investigation,Unsubstantiated,Brooklyn,Brooklyn North
1,05/25/2018,1,2,Force,Beat,CCRB,2000,2817,False,False,Full Investigation,Unsubstantiated,Full Investigation,Unsubstantiated,Brooklyn,Brooklyn North
2,05/25/2018,1,3,Force,Beat,CCRB,2000,2817,False,False,Full Investigation,Unsubstantiated,Full Investigation,Unsubstantiated,Brooklyn,Brooklyn North
3,05/25/2018,2,1,Discourtesy,Curse,CCRB,2000,2433,False,False,Truncated,Complaint Withdrawn,Truncated,Complaint Withdrawn,Manhattan,Manhattan North
4,05/25/2018,3,1,Discourtesy,Curse,CCRB,2000,2445,False,False,Other,Miscellaneous,Other,Miscellaneous,Manhattan,Other


In [10]:
complaintsData.columns

Index(['Extract Run Date', 'UniqueRandomComplaintId', 'Allegation Id',
       'Allegation FADO Type', 'Allegation', 'Case Type', 'Closed Year',
       'Days From Received Date To Case Closing', 'Re-opened',
       'Ever On DA Hold', 'Complaint Outcome Path', 'Complaint Disposition',
       'Allegation Outcome Path', 'Allegation Disposition',
       'Borough Of Incident', 'Patrol Borough Of Incident'],
      dtype='object')

In [11]:
complaintsData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309260 entries, 0 to 309259
Data columns (total 16 columns):
Extract Run Date                           309260 non-null object
UniqueRandomComplaintId                    309260 non-null int64
Allegation Id                              309260 non-null int64
Allegation FADO Type                       309260 non-null object
Allegation                                 309257 non-null object
Case Type                                  309260 non-null object
Closed Year                                309260 non-null int64
Days From Received Date To Case Closing    309260 non-null int64
Re-opened                                  309260 non-null bool
Ever On DA Hold                            309260 non-null bool
Complaint Outcome Path                     309260 non-null object
Complaint Disposition                      309260 non-null object
Allegation Outcome Path                    309260 non-null object
Allegation Disposition                 

In [12]:
complaintsData.isna().sum()

Extract Run Date                             0
UniqueRandomComplaintId                      0
Allegation Id                                0
Allegation FADO Type                         0
Allegation                                   3
Case Type                                    0
Closed Year                                  0
Days From Received Date To Case Closing      0
Re-opened                                    0
Ever On DA Hold                              0
Complaint Outcome Path                       0
Complaint Disposition                        0
Allegation Outcome Path                      0
Allegation Disposition                       0
Borough Of Incident                        710
Patrol Borough Of Incident                   0
dtype: int64

In [13]:
complaintsData['Borough Of Incident'].fillna('Missing', inplace = True)
complaintsData['Allegation'].fillna('Missing', inplace = True)

In [14]:
columns = list(complaintsData.select_dtypes('object').columns)
for col in columns:
    print("--------------Value Counts for column: ",col)
    print(complaintsData[col].value_counts())

--------------Value Counts for column:  Extract Run Date
05/25/2018    309260
Name: Extract Run Date, dtype: int64
--------------Value Counts for column:  Allegation FADO Type
Abuse of Authority    148909
Force                  91280
Discourtesy            52993
Offensive Language      9068
OMN                     7010
Name: Allegation FADO Type, dtype: int64
--------------Value Counts for column:  Allegation
Physical force                            65521
Word                                      46563
Threat of arrest                          16155
Refusal to provide name/shield number     15286
Premises entered and/or searched          14367
Stop                                      13744
Search (of person)                        13501
Threat of force (verbal or physical)       9932
Question and/or stop                       9436
Frisk                                      7790
Vehicle search                             7174
Vehicle stop                               6900
Gun Pointed

##### We can drop columns Extract Run Date and Case Type as they only have a simgle value i.e no variance

In [15]:
complaintsData.drop(['Extract Run Date', 'Case Type'], axis = 1, inplace = True)

In [16]:
complaintsData.tail(30)

,UniqueRandomComplaintId,Allegation Id,Allegation FADO Type,Allegation,Closed Year,Days From Received Date To Case Closing,Re-opened,Ever On DA Hold,Complaint Outcome Path,Complaint Disposition,Allegation Outcome Path,Allegation Disposition,Borough Of Incident,Patrol Borough Of Incident
309230,104698,6,Abuse of Authority,Search (of person),2017,147,False,False,Full Investigation,Unsubstantiated,Full Investigation,Unsubstantiated,Brooklyn,Brooklyn South
309231,104698,7,Abuse of Authority,Frisk,2017,147,False,False,Full Investigation,Unsubstantiated,Full Investigation,Unsubstantiated,Brooklyn,Brooklyn South
309232,104698,8,Abuse of Authority,Search (of person),2017,147,False,False,Full Investigation,Unsubstantiated,Full Investigation,Unsubstantiated,Brooklyn,Brooklyn South
309233,104698,9,OMN,Failure to prepare a memo book entry,2017,147,False,False,Full Investigation,Unsubstantiated,Other,Other Misconduct,Brooklyn,Brooklyn South
309234,104698,10,OMN,Failure to prepare a memo book entry,2017,147,False,False,Full Investigation,Unsubstantiated,Other,Other Misconduct,Brooklyn,Brooklyn South
309235,104698,11,OMN,Failure to produce stop and frisk report,2017,147,False,False,Full Investigation,Unsubstantiated,Other,Other Misconduct,Brooklyn,Brooklyn South
309236,104698,12,Abuse of Authority,Threat of arrest,2017,147,False,False,Full Investigation,Unsubstantiated,Full Investigation,Unsubstantiated,Brooklyn,Brooklyn South
309237,104699,1,Force,Gun Pointed,2017,55,False,False,Truncated,Complainant Uncooperative,Truncated,Complainant Uncooperative,Brooklyn,Brooklyn North
309238,104699,2,Abuse of Authority,Stop,2017,55,False,False,Truncated,Complainant Uncooperative,Truncated,Complainant Uncooperative,Brooklyn,Brooklyn North
309239,104700,1,Force,Chokehold,2017,126,False,False,Full Investigation,Unsubstantiated,Full Investigation,Unsubstantiated,Brooklyn,Brooklyn South


In [17]:
max(complaintsData.UniqueRandomComplaintId), min(complaintsData.UniqueRandomComplaintId)

(104707, 1)

# Task 1 : Check if Re-opened and Borough Of Incident are related

In [18]:
table = pd.crosstab(index = complaintsData['Re-opened'], columns = complaintsData['Borough Of Incident'])

In [19]:
complaintsData['Borough Of Incident'].value_counts()

Brooklyn         107115
Bronx             72531
Manhattan         67190
Queens            47395
Staten Island     14023
Missing             710
Outside NYC         296
Name: Borough Of Incident, dtype: int64

In [20]:
complaintsData['Patrol Borough Of Incident'].value_counts()

Bronx              72293
Brooklyn North     61885
Brooklyn South     44571
Manhattan North    38863
Queens South       29226
Manhattan South    27330
Queens North       18040
Staten Island      13978
Other               3074
Name: Patrol Borough Of Incident, dtype: int64

In [21]:
complaintsData['Re-opened'].value_counts()

False    305599
True       3661
Name: Re-opened, dtype: int64

### Here, we can see that the two values 'Re-opened' and 'Borough Of Incident' are dependent for alpha = 0.05 and probability = 0.95

In [22]:
stat, p, dof, expected = chi2_contingency(table)
print(stat)
print(p)
print('dof=%d' % dof)
print(expected)

prob = 0.95
critical = chi2.ppf(prob, dof)
print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
if abs(stat) >= critical:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')

alpha = 1.0 - prob
print('significance=%.3f, p=%.10f' % (alpha, p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')

49.1693408841478
6.8961375469712494e-09
dof=6
[[7.16723827e+04 1.05846980e+05 6.63946091e+04 7.01595066e+02
  2.92495971e+02 4.68339410e+04 1.38569966e+04]
 [8.58617316e+02 1.26802048e+03 7.95390901e+02 8.40493436e+00
  3.50402897e+00 5.61058963e+02 1.66003373e+02]]
probability=0.950, critical=12.592, stat=49.169
Dependent (reject H0)
significance=0.050, p=0.0000000069
Dependent (reject H0)


## Task 2

In [23]:
complaintsData.shape

(309260, 14)

In [24]:
import numpy as np

In [25]:
complaintsData.head(50)

,UniqueRandomComplaintId,Allegation Id,Allegation FADO Type,Allegation,Closed Year,Days From Received Date To Case Closing,Re-opened,Ever On DA Hold,Complaint Outcome Path,Complaint Disposition,Allegation Outcome Path,Allegation Disposition,Borough Of Incident,Patrol Borough Of Incident
0,1,1,Force,Beat,2000,2817,False,False,Full Investigation,Unsubstantiated,Full Investigation,Unsubstantiated,Brooklyn,Brooklyn North
1,1,2,Force,Beat,2000,2817,False,False,Full Investigation,Unsubstantiated,Full Investigation,Unsubstantiated,Brooklyn,Brooklyn North
2,1,3,Force,Beat,2000,2817,False,False,Full Investigation,Unsubstantiated,Full Investigation,Unsubstantiated,Brooklyn,Brooklyn North
3,2,1,Discourtesy,Curse,2000,2433,False,False,Truncated,Complaint Withdrawn,Truncated,Complaint Withdrawn,Manhattan,Manhattan North
4,3,1,Discourtesy,Curse,2000,2445,False,False,Other,Miscellaneous,Other,Miscellaneous,Manhattan,Other
5,3,2,Offensive Language,Other - Ethnic Slur,2000,2445,False,False,Other,Miscellaneous,Other,Miscellaneous,Manhattan,Other
6,4,1,Discourtesy,Word,2000,490,False,False,Full Investigation,Unfounded,Full Investigation,Unfounded,Queens,Queens North
7,4,2,Abuse of Authority,Property damaged,2000,490,False,False,Full Investigation,Unfounded,Full Investigation,Unfounded,Queens,Queens North
8,4,3,Force,Physical force,2000,490,False,False,Full Investigation,Unfounded,Full Investigation,Unfounded,Queens,Queens North
9,5,1,Discourtesy,Curse,2000,511,False,False,Full Investigation,Unsubstantiated,Full Investigation,Unsubstantiated,Queens,Queens South


In [26]:
complaintsData = pd.get_dummies(complaintsData)

In [27]:
complaintsData.head()

,UniqueRandomComplaintId,Allegation Id,Closed Year,Days From Received Date To Case Closing,Re-opened,Ever On DA Hold,Allegation FADO Type_Abuse of Authority,Allegation FADO Type_Discourtesy,Allegation FADO Type_Force,Allegation FADO Type_OMN,...,Borough Of Incident_Staten Island,Patrol Borough Of Incident_Bronx,Patrol Borough Of Incident_Brooklyn North,Patrol Borough Of Incident_Brooklyn South,Patrol Borough Of Incident_Manhattan North,Patrol Borough Of Incident_Manhattan South,Patrol Borough Of Incident_Other,Patrol Borough Of Incident_Queens North,Patrol Borough Of Incident_Queens South,Patrol Borough Of Incident_Staten Island
0,1,1,2000,2817,False,False,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
1,1,2,2000,2817,False,False,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
2,1,3,2000,2817,False,False,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
3,2,1,2000,2433,False,False,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,3,1,2000,2445,False,False,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0


In [28]:
X_train, X_test, Y_train, Y_test = train_test_split(complaintsData.drop(['Days From Received Date To Case Closing'], axis = 1),
                                                   complaintsData['Days From Received Date To Case Closing'],
                                                   test_size = 0.2, random_state = 42)

In [29]:
for val in X_train, X_test, Y_train, Y_test:
    print(val.shape)

(247408, 221)
(61852, 221)
(247408,)
(61852,)


In [30]:
rnd_reg = RandomForestRegressor(max_depth = 10)

In [31]:
rnd_reg.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=10, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [32]:
preds = rnd_reg.predict(X_test)

In [33]:
for i in zip(preds, Y_test):
    print(i)

(343.9835593722486, 419)
(390.19264517606166, 414)
(281.59460186413116, 629)
(129.906213894511, 132)
(302.6955784037924, 441)
(115.49510855732726, 190)
(87.68377009980891, 87)
(346.10297587902505, 295)
(350.5076927703857, 273)
(170.68162725315432, 131)
(115.49510855732726, 151)
(340.060484407043, 384)
(345.53138827399926, 183)
(114.39903467275788, 45)
(125.48753948329878, 92)
(281.7172142881834, 422)
(122.89178560956472, 72)
(125.10928797032363, 82)
(388.23906847558254, 380)
(344.5990032566798, 218)
(350.5076927703857, 523)
(293.54195987944144, 252)
(189.20698235474777, 189)
(102.73371978145633, 93)
(48.04271050612198, 50)
(115.49510855732726, 67)
(294.4256436005008, 195)
(125.10928797032363, 103)
(51.30213829802617, 60)
(378.3919896639667, 300)
(349.0271097731896, 567)
(115.49510855732726, 162)
(294.04901514888104, 217)
(115.49510855732726, 152)
(284.47097745131794, 531)
(346.10297587902505, 224)
(50.52092657449606, 43)
(243.54460074983191, 286)
(283.5717227296989, 128)
(274.849307949

(140.06418108594207, 153)
(115.49510855732726, 88)
(115.49510855732726, 180)
(321.8216973392015, 176)
(120.72830528212386, 110)
(319.6490565006714, 251)
(184.28736879367784, 216)
(89.55590966815076, 57)
(114.52281339326947, 77)
(125.88450511082378, 130)
(346.10297587902505, 232)
(281.0179681834809, 259)
(270.51033254177696, 270)
(291.6611874579975, 132)
(125.10928797032363, 89)
(129.40837098788353, 126)
(115.49510855732726, 79)
(114.52281339326947, 74)
(385.21873153683015, 258)
(281.59460186413116, 143)
(387.32745645889554, 260)
(114.52281339326947, 64)
(346.10297587902505, 332)
(351.54185209851397, 315)
(356.43207347903376, 398)
(287.5618571751033, 286)
(388.23906847558254, 360)
(101.98313697956502, 61)
(115.35176562790244, 114)
(282.6164690098026, 343)
(114.39903467275788, 67)
(346.503611659327, 134)
(346.10297587902505, 265)
(342.0988947679664, 254)
(346.10297587902505, 388)
(301.2975135181695, 264)
(356.4921694769604, 346)
(283.5126882966704, 257)
(354.40896013103986, 430)
(388.239

(51.45902127517386, 41)
(115.49510855732726, 96)
(118.88978641610727, 93)
(314.30796672113047, 299)
(115.35176562790244, 120)
(347.86511227559095, 513)
(114.52281339326947, 57)
(351.6058928850208, 190)
(346.10297587902505, 594)
(157.62137894423049, 216)
(165.39787031080385, 108)
(304.4020079388615, 450)
(115.49510855732726, 111)
(125.10928797032363, 132)
(333.8959261898225, 242)
(355.16539841778774, 508)
(414.6723254894485, 441)
(115.49510855732726, 50)
(342.0988947679664, 323)
(176.49004090388746, 144)
(115.49510855732726, 86)
(115.49510855732726, 87)
(166.7275451890786, 178)
(302.51041874682903, 202)
(102.38474890989335, 113)
(378.6805752474952, 270)
(283.5717227296989, 338)
(305.00513886001846, 552)
(263.545755604031, 102)
(114.39903467275788, 68)
(346.10297587902505, 392)
(115.35176562790244, 124)
(342.0988947679664, 202)
(300.64646015041933, 429)
(123.97298128468115, 154)
(115.49510855732726, 98)
(166.46455805743105, 219)
(115.49510855732726, 49)
(235.60551130361324, 271)
(148.883

(115.49510855732726, 90)
(106.12194944681559, 91)
(115.35176562790244, 69)
(294.05724816410856, 392)
(103.63716813928748, 29)
(165.8364163349884, 194)
(284.47097745131794, 134)
(115.49510855732726, 172)
(286.96569756450737, 496)
(48.003148747600086, 49)
(343.9835593722486, 371)
(345.6837677233671, 389)
(115.49510855732726, 85)
(115.49510855732726, 86)
(346.10297587902505, 233)
(123.97298128468115, 118)
(115.49510855732726, 81)
(115.49510855732726, 100)
(115.49510855732726, 72)
(215.02314439564833, 127)
(115.49510855732726, 117)
(412.8518349093214, 374)
(103.16199001229435, 71)
(357.90614971108016, 387)
(252.63144698561285, 206)
(345.6837677233671, 278)
(342.0988947679664, 488)
(115.49510855732726, 69)
(115.49510855732726, 78)
(115.49510855732726, 142)
(333.8959261898225, 447)
(115.49510855732726, 88)
(295.1314198666265, 609)
(352.0818552568086, 569)
(115.49510855732726, 69)
(115.35176562790244, 149)
(395.2248496570087, 414)
(256.33478400048745, 174)
(114.52281339326947, 221)
(223.36463

(303.01819944476125, 232)
(114.52281339326947, 72)
(275.26768263508006, 223)
(125.10928797032363, 115)
(302.51041874682903, 230)
(353.3129795440795, 356)
(346.10297587902505, 166)
(115.49510855732726, 115)
(409.25177617431063, 439)
(345.06023890677164, 421)
(115.49510855732726, 112)
(115.49510855732726, 118)
(50.347404084625076, 62)
(284.47097745131794, 302)
(123.90045947459151, 100)
(345.6837677233671, 206)
(373.5211741698035, 257)
(125.10928797032363, 110)
(272.9688176758448, 218)
(129.56352648967325, 147)
(357.85411783863617, 618)
(99.61929458121547, 75)
(115.35176562790244, 85)
(123.90045947459151, 161)
(355.16539841778774, 360)
(48.23616045840839, 40)
(281.59460186413116, 204)
(107.2571283193204, 112)
(391.08426946481705, 231)
(115.49510855732726, 90)
(346.503611659327, 289)
(134.9398858923869, 85)
(123.90045947459151, 108)
(168.09664424995586, 181)
(280.69534714251205, 354)
(156.13807477428801, 14)
(49.7979040097685, 145)
(107.80901173748039, 93)
(287.85727630848925, 393)
(367.54

...

(175.56991799295008, 153)
(250.4937293515804, 188)
(313.74447957143786, 492)
(372.13220459912964, 461)
(346.10297587902505, 348)
(115.35176562790244, 90)
(125.8803427884928, 107)
(115.49510855732726, 117)
(363.1052411560944, 317)
(337.58018666672723, 398)
(313.74447957143786, 231)
(301.2975135181695, 312)
(293.54195987944144, 317)
(115.49510855732726, 115)
(125.10928797032363, 88)
(123.14038416767326, 89)
(354.072536347908, 583)
(133.65107890836106, 128)
(101.98313697956502, 133)
(400.61527327520287, 332)
(524.4084283975393, 642)
(356.87217351304093, 338)
(356.43207347903376, 444)
(114.52281339326947, 65)
(115.49510855732726, 171)
(115.49510855732726, 142)
(346.10297587902505, 391)
(156.6716723507758, 426)
(388.23906847558254, 431)
(115.49510855732726, 61)
(348.38827626360916, 161)
(115.49510855732726, 79)
(394.7215782768526, 504)
(295.4475107461722, 320)
(312.56068105124564, 329)
(346.10297587902505, 518)
(115.49510855732726, 72)
(346.10297587902505, 391)
(301.7190359207024, 446)
(300

(125.10928797032363, 127)
(247.17814877896, 205)
(48.04271050612198, 40)
(346.10297587902505, 111)
(355.6108705907105, 289)
(351.1290734678093, 514)
(49.529534014369325, 46)
(346.10297587902505, 420)
(115.49510855732726, 167)
(393.4631441474126, 307)
(115.49510855732726, 77)
(140.3924774134126, 168)
(114.52281339326947, 85)
(404.8550326534226, 455)
(346.10297587902505, 414)
(553.7017921764021, 477)
(129.906213894511, 114)
(123.14038416767326, 111)
(332.56569294919393, 366)
(276.54244332885696, 244)
(346.10297587902505, 462)
(303.90164939104744, 325)
(115.49510855732726, 135)
(165.8364163349884, 183)
(300.6214956959095, 453)
(354.072536347908, 168)
(101.98313697956502, 56)
(115.49510855732726, 106)
(125.10928797032363, 84)
(342.0988947679664, 275)
(49.243882039221226, 42)
(282.3821841339117, 275)
(73.2253097087399, 72)
(115.49510855732726, 68)
(147.40394116999943, 125)
(102.73371978145633, 170)
(346.10297587902505, 289)
(339.4543970577826, 431)
(101.98313697956502, 70)
(115.495108557327

(115.49510855732726, 95)
(313.8001860231982, 290)
(149.86889624783973, 220)
(161.11481274347122, 166)
(115.49510855732726, 147)
(125.10928797032363, 187)
(417.54935759794853, 546)
(222.3725509591061, 203)
(115.49510855732726, 62)
(387.32745645889554, 466)
(115.35176562790244, 75)
(125.10928797032363, 188)
(115.49510855732726, 78)
(283.8943437706677, 159)
(295.46182953214935, 122)
(167.26320039248367, 148)
(353.44864839125484, 385)
(115.49510855732726, 63)
(115.49510855732726, 75)
(114.39903467275788, 57)
(390.19264517606166, 393)
(102.38474890989335, 286)
(273.54812437536185, 306)
(372.13220459912964, 162)
(371.24696030977526, 441)
(301.2975135181695, 272)
(300.1137149979773, 39)
(115.49510855732726, 175)
(295.9241407367282, 190)
(400.73476775076483, 367)
(86.16731472816123, 79)
(123.97298128468115, 183)
(115.49510855732726, 122)
(281.59460186413116, 365)
(115.49510855732726, 63)
(428.5281122591109, 531)
(115.35176562790244, 93)
(114.52281339326947, 84)
(115.49510855732726, 95)
(353.11

(149.1780121582238, 120)
(114.52281339326947, 105)
(464.299509035602, 437)
(107.80901173748039, 70)
(101.0638787783459, 49)
(115.49510855732726, 250)
(333.8959261898225, 173)
(94.93449572618006, 92)
(388.5481889502655, 463)
(284.47097745131794, 392)
(115.49510855732726, 72)
(347.8289539493903, 301)
(115.49510855732726, 78)
(114.39903467275788, 70)
(115.49510855732726, 71)
(30.903347262807653, 26)
(320.92875763179035, 317)
(257.0373636125094, 225)
(282.1023825620634, 295)
(102.41377660027071, 99)
(344.9054839291588, 450)
(148.88372790126274, 144)
(333.8959261898225, 75)
(409.0927515512716, 448)
(422.2076063736871, 486)
(352.5578973157484, 369)
(97.22921711177767, 94)
(125.10928797032363, 311)
(342.0988947679664, 351)
(125.10928797032363, 112)
(98.16594582100328, 48)
(358.22657169303403, 548)
(61.70492147406001, 52)
(115.49510855732726, 52)
(102.89301314368818, 105)
(342.0988947679664, 567)
(34.2257152653252, 8)
(388.23906847558254, 513)
(297.8272032888237, 161)
(284.47097745131794, 147)

(301.04102108487683, 426)
(345.20631954267947, 320)
(123.90045947459151, 73)
(123.14038416767326, 143)
(281.0179681834809, 105)
(115.49510855732726, 87)
(115.49510855732726, 112)
(322.6598129506864, 325)
(101.98313697956502, 124)
(342.0988947679664, 141)
(349.0271097731896, 420)
(65.80473885414811, 51)
(50.43769174265987, 24)
(115.49510855732726, 99)
(342.0988947679664, 130)
(115.49510855732726, 182)
(125.10928797032363, 80)
(301.2975135181695, 332)
(115.49510855732726, 126)
(357.88917784090495, 420)
(115.49510855732726, 104)
(370.3188889331101, 329)
(473.0031104009861, 499)
(51.13863276867479, 59)
(287.64401543856184, 295)
(374.5670633799637, 219)
(148.07964178884703, 132)
(115.49510855732726, 103)
(115.49510855732726, 80)
(115.49510855732726, 99)
(115.49510855732726, 97)
(233.81693094723934, 150)
(258.457270286192, 533)
(115.49510855732726, 77)
(115.35176562790244, 96)
(27.420752166874763, 9)
(115.49510855732726, 64)
(282.1023825620634, 294)
(414.3643165863331, 499)
(115.495108557327

In [34]:
rnd_reg.score(X_test, Y_test)

0.6430007842687034

In [35]:
output = sorted(zip(X_train.columns, rnd_reg.feature_importances_), key=lambda x: x[1])

In [36]:
type(output)

list

##### These are some of the features from the sorted list in order from lowest feature importance to highest feature importance 

In [37]:
for i in output:
    print(i)

('Allegation_Abuse of Authority', 0.0)
('Allegation_Arrest/D. A. T.', 0.0)
('Allegation_Black', 0.0)
('Allegation_Discourtesy', 0.0)
('Allegation_Failure to document strip search', 0.0)
('Allegation_Flashlight As Club', 0.0)
('Allegation_Forcible Removal to Hospital', 0.0)
('Allegation_Gay/Lesbian Slur', 0.0)
('Allegation_Gun As Club', 0.0)
('Allegation_Improper dissemination of medical info', 0.0)
('Allegation_Jewish', 0.0)
('Allegation_Missing', 0.0)
('Allegation_Nightstick/Billy/Club', 0.0)
('Allegation_Oriental', 0.0)
('Allegation_Physical disability', 0.0)
('Allegation_Profane Gesture', 0.0)
('Allegation_Property Seized', 0.0)
('Allegation_Radio As Club', 0.0)
('Allegation_Restricted Breathing', 0.0)
('Allegation_Search of recording device', 0.0)
('Allegation_Sexist Remark', 0.0)
('Allegation_Threat re: removal to hospital', 0.0)
('Allegation_Threat to Property', 0.0)
('Allegation_Vehicle Searched', 0.0)
('Allegation_White', 0.0)
('Complaint Disposition_Conciliated', 0.0)
('Compla

## Task 4

In [14]:
complaintsData.head()

,UniqueRandomComplaintId,Allegation Id,Closed Year,Days From Received Date To Case Closing,Re-opened,Ever On DA Hold,Allegation FADO Type_Abuse of Authority,Allegation FADO Type_Discourtesy,Allegation FADO Type_Force,Allegation FADO Type_OMN,...,Borough Of Incident_Staten Island,Patrol Borough Of Incident_Bronx,Patrol Borough Of Incident_Brooklyn North,Patrol Borough Of Incident_Brooklyn South,Patrol Borough Of Incident_Manhattan North,Patrol Borough Of Incident_Manhattan South,Patrol Borough Of Incident_Other,Patrol Borough Of Incident_Queens North,Patrol Borough Of Incident_Queens South,Patrol Borough Of Incident_Staten Island
0,1,1,2000,2817,False,False,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
1,1,2,2000,2817,False,False,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
2,1,3,2000,2817,False,False,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
3,2,1,2000,2433,False,False,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,3,1,2000,2445,False,False,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0


In [15]:
complaintsData['Re-opened'].value_counts()

False    305599
True       3661
Name: Re-opened, dtype: int64

In [45]:
X_train, X_test, Y_train, Y_test = train_test_split(complaintsData.drop(['Re-opened'], axis = 1), 
                                                    complaintsData['Re-opened'],
                                                    test_size = 0.2, random_state = 42)

In [56]:
for val in [X_train, X_test, Y_train, Y_test]:
    print(val.shape)
    

(247408, 221)
(61852, 221)
(247408,)
(61852,)


In [48]:
Y_train.value_counts()

False    244503
True       2905
Name: Re-opened, dtype: int64

In [77]:
trainData = pd.concat([X_train, Y_train], axis = 1)

In [78]:
trainData.shape

(247408, 222)

In [79]:
trainData.reset_index(inplace=True, drop=True)

In [80]:
dataUnderSampled = trainData[trainData['Re-opened'] == False].sample(n = 15000)

In [81]:
dataUnderSampled.head()

,UniqueRandomComplaintId,Allegation Id,Closed Year,Days From Received Date To Case Closing,Ever On DA Hold,Allegation FADO Type_Abuse of Authority,Allegation FADO Type_Discourtesy,Allegation FADO Type_Force,Allegation FADO Type_OMN,Allegation FADO Type_Offensive Language,...,Patrol Borough Of Incident_Bronx,Patrol Borough Of Incident_Brooklyn North,Patrol Borough Of Incident_Brooklyn South,Patrol Borough Of Incident_Manhattan North,Patrol Borough Of Incident_Manhattan South,Patrol Borough Of Incident_Other,Patrol Borough Of Incident_Queens North,Patrol Borough Of Incident_Queens South,Patrol Borough Of Incident_Staten Island,Re-opened
113173,88875,3,2014,361,False,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,False
143113,91150,4,2015,124,False,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,False
224567,72547,1,2011,264,False,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,False
215858,78527,2,2012,216,False,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,False
34259,33366,2,2006,161,False,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,False


In [82]:
dataUnderSampled = pd.concat([dataUnderSampled, trainData[trainData['Re-opened'] == True]], axis = 0)

In [83]:
dataUnderSampled.shape

(17905, 222)

In [84]:
dataUnderSampled.reset_index(drop=True, inplace = True)

In [85]:
dataUnderSampled['Re-opened'].value_counts()

False    15000
True      2905
Name: Re-opened, dtype: int64

In [86]:
X, Y = dataUnderSampled.drop(['Re-opened'], axis = 1), dataUnderSampled['Re-opened']
                                                   

In [87]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_res, Y_res = sm.fit_resample(X, Y)

In [88]:
Y_res.value_counts()

True     15000
False    15000
Name: Re-opened, dtype: int64

In [174]:
rnd_clf = RandomForestClassifier(max_depth = 60)

In [175]:
rnd_clf.fit(X_res, Y_res)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=60, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [176]:
print(classification_report(Y_test, rnd_clf.predict(X_test)))

              precision    recall  f1-score   support

       False       0.99      0.97      0.98     61096
        True       0.17      0.56      0.26       756

    accuracy                           0.96     61852
   macro avg       0.58      0.77      0.62     61852
weighted avg       0.98      0.96      0.97     61852



In [130]:
# 70
#               precision    recall  f1-score   support

#        False       0.99      0.97      0.98     61096
#         True       0.17      0.55      0.26       756

#     accuracy                           0.96     61852
#    macro avg       0.58      0.76      0.62     61852
# weighted avg       0.98      0.96      0.97     61852
